一、Pytorch基本操作考察

In [9]:
#调用torch库
import torch
import torchvision
import torchvision.transforms as transforms
#调用numpy库
import numpy as np
#调用matplotlib库
import matplotlib.pyplot as plt
#调用random库
import random

问题1. 使用 𝐓𝐞𝐧𝐬𝐨𝐫 初始化一个 𝟏 × 𝟑 的矩阵 𝑴 和一个 𝟐 × 𝟏 的矩阵 𝑵，对两矩阵进行减法操作（要求
实现三种不同的形式），给出结果并分析三种方式的不同（如果出现报错，分析报错的原因），
同时需要指出在计算过程中发生了什么

In [11]:
M = torch.rand(1, 3)
N = torch.rand(2, 1)
print(M.shape, N.shape)
print(M)
print(N)

torch.Size([1, 3]) torch.Size([2, 1])
tensor([[0.4942, 0.2690, 0.1761]])
tensor([[0.4602],
        [0.2456]])


形式1 直接相加
此处使用了tensor的广播机制，二者自动复制成为相同的shape进行运算，因此可以正常运行不会报错

In [12]:
Add = M + N
print(Add.shape)
print(Add)

torch.Size([2, 3])
tensor([[0.9545, 0.7292, 0.6364],
        [0.7399, 0.5147, 0.4218]])


形式2 使用Pytorch提供的加和函数进行运算，原理同上
依然使用了tensor的广播机制，二者自动复制成为相同的shape进行运算，因此可以正常运行不会报错

In [13]:
Add = torch.add(M, N)
print(Add.shape)
print(Add)

torch.Size([2, 3])
tensor([[0.9545, 0.7292, 0.6364],
        [0.7399, 0.5147, 0.4218]])


形式3 使用tensor自身的inplace操作
这一步不会进行新tensor的产生，也不能在原tensor的内存空间上进行广播，因此要求两tensor具备相同的shape，shape不同则会报错

In [14]:
Add = M.add_(N)
print(Add.shape)
print(Add)

RuntimeError: output with shape [1, 3] doesn't match the broadcast shape [2, 3]

问题2-1. 利用 𝐓𝐞𝐧𝐬𝐨𝐫 创建两个大小分别 𝟑 × 𝟐 和 𝟒 × 𝟐 的随机数矩阵 𝑷 和 𝑸 ，要求服从均值为0，标
准差0.01为的正态分布；

In [45]:
P = torch.normal(mean=torch.full((3,2),0.0),std=torch.full((3,2),0.01))
Q = torch.normal(mean=torch.full((4,2),0.0),std=torch.full((4,2),0.01))
print(P.shape, Q.shape)
print(P)
print(Q)

torch.Size([3, 2]) torch.Size([4, 2])
tensor([[-0.0045,  0.0189],
        [ 0.0087, -0.0015],
        [ 0.0048, -0.0021]])
tensor([[-0.0022,  0.0085],
        [-0.0053,  0.0062],
        [ 0.0136, -0.0009],
        [-0.0035, -0.0029]])


问题2-2. 对第二步得到的矩阵 𝑸 进行形状变换得到 𝑸 的转置；

In [46]:
Q = Q.t()
print(Q.shape)
print(Q)

torch.Size([2, 4])
tensor([[-0.0022, -0.0053,  0.0136, -0.0035],
        [ 0.0085,  0.0062, -0.0009, -0.0029]])


问题2-3. 对上述得到的矩阵 𝑷 和矩阵 𝑸𝑻 求内积

In [48]:
R = torch.mm(P, Q)
print(R.shape)
print(R)

torch.Size([3, 4])
tensor([[ 1.7130e-04,  1.4187e-04, -7.8602e-05, -3.9020e-05],
        [-3.2270e-05, -5.5599e-05,  1.2048e-04, -2.5948e-05],
        [-2.8690e-05, -3.8415e-05,  6.6795e-05, -1.0365e-05]])


问题3. 给定公式 𝑦3 = 𝑦1 + 𝑦2 = 𝑥
2 + 𝑥
3
，且 𝑥 = 1。利用学习所得到的Tensor的相关知识，求𝑦3对𝑥的
梯度，即𝑑𝑦3
𝑑𝑥 。
要求在计算过程中，在计算 𝑥
3 时中断梯度的追踪，观察结果并进行原因分析

首先在第一次尝试中不中断y2的梯度追踪使torch的自动梯度追踪能够正常运行

In [55]:
x = torch.tensor([1.])
x = torch.autograd.Variable(x,requires_grad=True)
y1 = torch.pow(x, 2)
y2 = torch.pow(x, 3)
y3 = y1 + y2
y3.backward()
print(x.grad)

tensor([5.])


其次在第二次尝试中中断y2的梯度追踪，该尝试会导致梯度回传计算错误

In [56]:
x = torch.tensor([1.])
x = torch.autograd.Variable(x,requires_grad=True)
y1 = torch.pow(x, 2)
with torch.no_grad():
    y2 = torch.pow(x, 3)
y3 = y1 + y2
y3.backward()
print(x.grad)

tensor([2.])


该结果与第一次的计算出现差异，原因是y2变量的梯度计算未包含在内。而该变量的梯度值为3，为两次y3梯度值的计算差异。这证明了中断中间变量的梯度计算会导致最终的梯度计算出现问题。